In [2]:
import sys
sys.path.append('/Users/erikweilandt/Library/Caches/pypoetry/virtualenvs/tepem-FeojsIC2-py3.11/lib/python3.11/site-packages')

In [3]:
import numpy as np

In [4]:
num_slice = 4

In [5]:
x = np.linspace(0, 10, 2* num_slice + 1)
x

array([ 0.  ,  1.25,  2.5 ,  3.75,  5.  ,  6.25,  7.5 ,  8.75, 10.  ])

In [6]:
y = np.array([foo*2 for foo in x ])
y

array([ 0. ,  2.5,  5. ,  7.5, 10. , 12.5, 15. , 17.5, 20. ])

In [7]:
total = np.vstack((x,y)).T
total

array([[ 0.  ,  0.  ],
       [ 1.25,  2.5 ],
       [ 2.5 ,  5.  ],
       [ 3.75,  7.5 ],
       [ 5.  , 10.  ],
       [ 6.25, 12.5 ],
       [ 7.5 , 15.  ],
       [ 8.75, 17.5 ],
       [10.  , 20.  ]])

In [8]:
ltg = np.arange(0, 2*num_slice+1)
ltg

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [9]:
ltg2 = np.arange(2*num_slice+1, 4*num_slice+2)
ltg2

array([ 9, 10, 11, 12, 13, 14, 15, 16, 17])

In [27]:
mod = np.ones(2*num_slice+1, dtype= int)
for i, num in enumerate(mod, 1):
    if i%2==0:
        mod[i] +=1
    mod[-1] = 1
ltg_edit = np.repeat(ltg, mod)#.reshape((num_slice, 3))
ltg_edit, mod

(array([0, 1, 2, 2, 3, 4, 4, 5, 6, 6, 7, 8]),
 array([1, 1, 2, 1, 2, 1, 2, 1, 1]))

In [11]:
ltg_edit2 = np.repeat(ltg2, mod).reshape((num_slice, 3))
ltg_edit2

array([[ 9, 10, 11],
       [11, 12, 13],
       [13, 14, 15],
       [15, 16, 17]])

In [12]:
for slab in ltg_edit:
    print(slab)

[0 1 2]
[2 3 4]
[4 5 6]
[6 7 8]


# LTG Map

In [13]:
def generate_ltg(
    num_slabs: int, fe_order, velocity_ltg: bool = False
) :
    num_dof_rhs = fe_order[1]
    if velocity_ltg:
        num_dof_rhs = num_dof_rhs - 2
    num_loc_dof = (fe_order[0] + 1) * (num_dof_rhs + 1)
    ltg = np.arange(num_loc_dof * num_slabs).reshape((num_slabs, num_loc_dof))
    for i, _ in enumerate(ltg):
        ltg[i] = ltg[i] - (num_dof_rhs+1) * i
    return ltg
test = generate_ltg(3, (2,2))#, velocity_ltg=True)
test

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8],
       [ 6,  7,  8,  9, 10, 11, 12, 13, 14],
       [12, 13, 14, 15, 16, 17, 18, 19, 20]])

In [14]:
lt = [1,2,3,4,5]
lt_a = np.array(lt).T
lt_a[4:]

array([5])

In [15]:
NU = 100
PRESSURE_GRAD = -1
C_CONST = 1 / 2 * 1 / NU * (-PRESSURE_GRAD)
RADIUS = 1
LENGTH = 10


def g_1(x_1: float, x_2: float) -> float:
    if x_1 < 1e-14:
        return C_CONST * (RADIUS - x_2) * (x_2 + RADIUS)
    else:
        return 0

# chebychev points

In [16]:
def get_chebby_points(start, end, num):
    ref_points = [-np.cos(np.pi * i / num) for i in range(num+1)]
    interval_points = [map_min1plus1_to_ab(x, start, end) for x in ref_points]
    return np.array(interval_points)

def map_min1plus1_to_ab(x,a, b):
    return (b-a)/2 * x + (a+b)/2

get_chebby_points(-1,1,4)

array([-1.00000000e+00, -7.07106781e-01, -6.12323400e-17,  7.07106781e-01,
        1.00000000e+00])

# high resolution plot

In [25]:
nx, ny = (3, 2)
x = np.linspace(0, 1, nx)
y = np.linspace(0, 1, ny)
xv, yv = np.meshgrid(x, y)
xv.flatten(), yv.flatten()
xv



array([[0. , 0.5, 1. ],
       [0. , 0.5, 1. ]])

In [24]:
test = [1,2,3,4,5,6]
test = np.array(test)
test.reshape((3,2))
test

array([1, 2, 3, 4, 5, 6])